In [7]:
from streetlevel import streetview

pano = streetview.find_panorama(-37.81246, 144.96514,radius =50)
print(pano.id)
print(pano.lat, pano.lon)
print(pano.date)

3yVCHQze4Glc1iTNVcMnUA
-37.81242916985612 144.9651631896737
2022-05


In [11]:
import math
from pathlib import Path

import geopandas as gpd
from shapely.geometry import box
from tqdm import tqdm
from streetlevel import streetview

# ==== 配置区 ====
shapefile_path = "test/test.shp"   # ← 替换为你的 Shapefile 路径
zoom = 17                          # Slippy Tile zoom level（约150m×150m 网格）
output_sh  = "download_sv.sh"      # 生成的 Bash 脚本

# ==== 工具函数 ====
def deg2tile(lat_deg: float, lon_deg: float, zoom: int):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int(
        (1.0 - math.log(math.tan(lat_rad) + 1.0 / math.cos(lat_rad)) / math.pi) / 2.0 * n
    )
    return xtile, ytile

def tile2deg(x: int, y: int, zoom: int):
    n = 2.0 ** zoom
    lon_deg = x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    return math.degrees(lat_rad), lon_deg

def tile_bounds(x: int, y: int, zoom: int):
    lat1, lon1 = tile2deg(x, y, zoom)
    lat2, lon2 = tile2deg(x + 1, y + 1, zoom)
    return box(lon1, lat2, lon2, lat1)

# ==== 1. 读取并合并边界面 ====
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")
boundary = gdf.geometry.unary_union

# ==== 2. 计算需遍历的 Tile 范围 ====
minx, miny, maxx, maxy = boundary.bounds
x_min, y_max = deg2tile(miny, minx, zoom)
x_max, y_min = deg2tile(maxy, maxx, zoom)
print(f"Tile X range: {x_min}…{x_max}, Y range: {y_min}…{y_max}")

# ==== 3. 遍历所有 Tile，收集 pano_id ====
pano_ids = set()
for x in tqdm(range(x_min, x_max + 1), desc="Tiles X"):
    for y in range(y_min, y_max + 1):
        tb = tile_bounds(x, y, zoom)
        if not boundary.intersects(tb):
            continue
        try:
            panos = streetview.get_coverage_tile(x, y)
        except Exception as e:
            print(f"[Warning] tile ({x},{y}) failed: {e}")
            continue
        for p in panos:
            pano_ids.add(p.id)

print(f"✅ Found {len(pano_ids)} unique pano IDs")

# ==== 4. 生成下载脚本 (.sh) ====
panos_sorted = sorted(pano_ids)
total = len(panos_sorted)

with open(output_sh, "w", encoding="utf-8") as f:
    f.write("#!/usr/bin/env bash\n\n")
    # 允许使用者通过 $1 传入导出目录，默认 panoramas
    f.write('out_dir="${1:-panoramas}"\n')
    f.write('mkdir -p "$out_dir"\n')
    f.write('echo "Export directory: $out_dir"\n\n')
    f.write(f"total={total}\n")
    f.write('echo "Total panoramas to download: $total"\n')
    f.write("i=1\n\n")
    f.write("for pano in \\\n")
    # 每行一个 pano_id，末尾续行
    for pid in panos_sorted[:-1]:
        f.write(f"  {pid} \\\n")
    f.write(f"  {panos_sorted[-1]}; do\n")
    f.write('  printf "[%d/%d] Downloading pano ID: %s\\n" "$i" "$total" "$pano"\n')
    f.write('  php ./streeview-dl.php "$pano"\n')
    # 下载完后，移动对应的三种文件到目标目录
    f.write('  mv stl-"$pano"*.jpg stm-"$pano"*.jpg sts-"$pano"*.jpg "$out_dir"/\n')
    f.write("  ((i++))\n")
    f.write("done\n")

# 设置脚本可执行权限（Unix 平台）
Path(output_sh).chmod(0o755)

print(f"[✓] Download script saved: {output_sh}")


/tmp/ipykernel_2386/992774747.py:38: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  boundary = gdf.geometry.unary_union


Tile X range: 118315…118316, Y range: 80426…80427


Tiles X: 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]

✅ Found 590 unique pano IDs
[✓] Download script saved: download_sv.sh
